# Create a Generative AI App That Uses Your Own Data (RAG)

## Lab Overview

**Retrieval Augmented Generation (RAG)** is a pattern that improves generative AI responses by grounding them in **custom data**. Instead of relying only on model training data, the application retrieves relevant documents and injects them into the prompt.

**Why RAG matters (AI‑102):**

* Reduces hallucinations
* Enables domain‑specific and enterprise chat apps
* Combines **search + generation**

**Estimated time:** ~45 minutes
**SDK status:** Pre‑release (APIs may change)

---

## High‑Level Architecture

**User → Chat App → Azure OpenAI (GPT‑4o)**
**                     ↘ Azure AI Search (Vector Index)**

Components:

* **GPT‑4o** – Generates natural‑language responses
* **Embedding model** – Converts text into vectors
* **Azure AI Search** – Stores and retrieves vectorized documents
* **Foundry hub + project** – Manages models, data, and indexes
* **Python client app** – Implements the RAG pattern

---

## 1. Create a Microsoft Foundry Hub and Project

> RAG features require a **Foundry hub–based project** (not a basic Foundry project).

### Steps

1. Open **[https://ai.azure.com](https://ai.azure.com)** and sign in
2. Go to **Management Center → All resources → Create new**
3. Select **Create new AI hub resource**

### Project Configuration

* **Project name:** Valid name
* **Hub:** Create new → Rename to a unique alphanumeric name
* **Advanced options:**

  * **Subscription:** Your Azure subscription
  * **Resource group:** Create or select
  * **Region:** East US 2 or Sweden Central

**Tip:** If *Create* is disabled, the hub name is not unique.

Wait for the project to finish provisioning.

---

## 2. Deploy Required Models

You need **two models** for RAG.

### A. Embedding Model (for vector search)

* **Model:** text‑embedding‑ada‑002
* **Deployment type:** Global Standard
* **Model version:** Default
* **TPM:** 50K (or max available)
* **Content filter:** DefaultV2

### B. Chat Model (for responses)

* **Model:** gpt‑4o
* **Deployment type:** Global Standard
* **Model version:** Most recent
* **TPM:** 50K

**Exam note:** Embeddings are used for **retrieval**, not generation.

---

## 3. Add Custom Data to the Project

### Data Description

* PDF travel brochures from *Margie’s Travel*

### Steps

1. Download and unzip:

   * [https://github.com/MicrosoftLearning/mslearn-ai-studio/raw/main/data/brochures.zip](https://github.com/MicrosoftLearning/mslearn-ai-studio/raw/main/data/brochures.zip)
2. In Foundry:

   * **My assets → Data + indexes → + New data**
3. Upload the **brochures** folder
4. Set data name to **brochures**

Wait for all PDFs to finish uploading.

---

## 4. Create a Vector Index (Azure AI Search)

Indexes enable **hybrid retrieval** (vector + keyword).

### Index Setup

* **Data source:** Data in Foundry → brochures
* **Azure AI Search:** Create new

  * Pricing tier: Basic
  * Location: Same as hub

### Vector Configuration

* **Index name:** brochures‑index
* **Vector search:** Enabled
* **Embedding model:** text‑embedding‑ada‑002
* **Embedding deployment:** Your embedding deployment

### Indexing Process

1. Crack PDFs
2. Chunk text
3. Embed chunks
4. Create search index
5. Register index

> Index creation may take several minutes.

---

## 5. Test the Index in the Playground

### Without Custom Data

1. Open **Playgrounds → Chat playground**
2. Prompt: *Where can I stay in New York?*
3. Result: Generic model response

### With Custom Data

1. In **Setup → Add your data**, select **brochures‑index**
2. Search type: **Hybrid (vector + keyword)**
3. Re‑submit the same prompt

Result: Response grounded in brochure content

---

## 6. Create a RAG Client Application (Python)

### Prepare Cloud Shell

1. Open **Azure Portal → Cloud Shell**
2. Choose **PowerShell (Classic)**
3. Clone the repo:

```
git clone https://github.com/microsoftlearning/mslearn-ai-studio mslearn-ai-foundry
cd mslearn-ai-foundry/labfiles/rag-app/python
```

### Install Dependencies

```
python -m venv labenv
./labenv/bin/Activate.ps1
pip install -r requirements.txt openai
```

---

## 7. Configure the Application

Edit `.env`:

| Variable        | Value                              |
| --------------- | ---------------------------------- |
| OPENAI_ENDPOINT | Azure OpenAI endpoint from Foundry |
| OPENAI_API_KEY  | API key from Foundry               |
| CHAT_MODEL      | gpt‑4o deployment name             |
| EMBEDDING_MODEL | embedding deployment name          |
| SEARCH_ENDPOINT | Azure AI Search endpoint           |
| SEARCH_API_KEY  | Azure AI Search key                |
| INDEX_NAME      | brochures‑index                    |

Save and exit the editor.

---

## 8. Understand the RAG Code Flow

The Python app:

1. Creates an Azure OpenAI client
2. Builds a system prompt (travel assistant)
3. Accepts user input
4. Generates embeddings for the query
5. Searches Azure AI Search for relevant chunks
6. Injects retrieved content into the prompt
7. Generates a grounded response
8. Displays source citations
9. Maintains chat history

---

## 9. Run the Application

```
python rag-app.py
```

### Example Prompts

* *Where should I go on vacation to see architecture?*
* *Where can I stay there?*

Type `quit` to exit.

---

## 10. Clean Up (Required)

To avoid Azure charges:

1. Open **Azure Portal → Resource groups**
2. Select the resource group used in this lab
3. Choose **Delete resource group**
4. Confirm deletion

---

## AI‑102 Exam Focus

* Purpose of **embeddings** in RAG
* Role of **Azure AI Search**
* Difference between **model‑only vs RAG** responses
* Why **vector search** is preferred
* Foundry **hub vs project** distinction
* How grounding reduces hallucinations

---

**This lab demonstrates the full RAG lifecycle: custom data → vector index → grounded generation → client app.**
